In [1]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB

In [2]:
import os
import openai
import numpy as np
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
from tqdm import tqdm

In [3]:
openai.api_key = ""

In [4]:
os.environ['OPENAI_API_KEY'] = ''

In [5]:
system_prompt = """
You are an expert news summarizer. When given a news article, your task is to condense the information into a brief summary, maintaining accuracy, neutrality, and clarity. Your summary should be 3-5 sentences long and focus on the key facts, events, and figures of the story. Include the essential who, what, where, when, and why, highlighting any significant impacts or next steps. Avoid subjective language, speculative information, and unnecessary background context. Keep the tone objective and concise.
"""

In [7]:
user_message = """

Title: Kristine exits PAR; new cyclone looms

Body:

MANILA, Philippines —  Severe Tropical Storm Kristine has left the Philippine area of responsibility, but another tropical cyclone is set to enter PAR over the weekend, according to the Philippine Atmospheric, Geophysical and Astronomical Services Administration (PAGASA).

Kristine left PAR at around 2 p.m. yesterday after battering Luzon over the past days, but it could make a U-turn toward the Philippine landmass tomorrow or Monday.

Despite Kristine’s exit, tropical cyclone wind signal No. 1 was still raised over Ilocos Norte, Ilocos Sur, La Union, Pangasinan, Apayao, Kalinga, Abra, Mountain Province, Ifugao, Benguet, Cagayan including Babuyan Islands, Isabela, Quirino, Nueva Vizcaya, Nueva Ecija, Tarlac, Zambales, Bataan, Pampanga, Bulacan, Metro Manila, the northern portion of Rizal and northern portion of Cavite.

Kristine was last monitored 410 kilometers west of Sinait, Ilocos Sur at 4 p.m., carrying maximum sustained winds of 95 km per hour near the center and gustiness of up to 115 kph.

It is expected to gradually intensify over the West Philippine Sea and could still be upgraded into a typhoon category.


PAGASA said Kristine will continue to move westward over the West Philippine Sea today and may loop counterclockwise while moving eastward toward PAR.

However, state meteorologists said this is still dependent on the behavior of another tropical cyclone that will enter PAR.

Another cyclone is set to enter PAR in Kristine’s wake as tropical storm Kong-rey, which will be named Leon once it enters PAR, was monitored 2,410 km east of Southeastern Luzon.

It was carrying maximum sustained winds of 65 kph and gustiness of up to 80 kph.

"""

In [8]:
struct = [{"role": "system", "content": system_prompt}]

struct.append({"role": "user", "content": user_message})
chat = openai.ChatCompletion.create(model="gpt-4o-mini",messages = struct)
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})
print("Assistant: " + response)

Assistant: Severe Tropical Storm Kristine has exited the Philippine area of responsibility but may potentially return to Philippine waters this weekend, as indicated by the Philippine Atmospheric, Geophysical and Astronomical Services Administration (PAGASA). As of yesterday, Kristine was located 410 kilometers west of Sinait, Ilocos Sur, with maximum sustained winds of 95 km/h. Signal No. 1 remains in effect for various regions, including parts of Luzon and Metro Manila. Concurrently, another tropical storm, Kong-rey, located 2,410 km east of Southeastern Luzon, is expected to enter the area soon and will be named Leon upon entry.


In [12]:
chat

<OpenAIObject chat.completion id=chatcmpl-AMUu9H3I6EdDZWp5FETRhq0cehCFq at 0x77fd0cf77c40> JSON: {
  "id": "chatcmpl-AMUu9H3I6EdDZWp5FETRhq0cehCFq",
  "object": "chat.completion",
  "created": 1729925377,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Severe Tropical Storm Kristine has exited the Philippine area of responsibility but may potentially return to Philippine waters this weekend, as indicated by the Philippine Atmospheric, Geophysical and Astronomical Services Administration (PAGASA). As of yesterday, Kristine was located 410 kilometers west of Sinait, Ilocos Sur, with maximum sustained winds of 95 km/h. Signal No. 1 remains in effect for various regions, including parts of Luzon and Metro Manila. Concurrently, another tropical storm, Kong-rey, located 2,410 km east of Southeastern Luzon, is expected to enter the area soon and will be named Leon upon entry.",
        "refusal": n

In [14]:
chat.usage.prompt_tokens

514